In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Path to your service account key file
KEY_PATH = "prefab-environs-454005-v7-b2f63bc9936c.json"

# Set up credentials
credentials = service_account.Credentials.from_service_account_file(KEY_PATH)

# Create BigQuery client
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# Example query
query = """
    SELECT *
    FROM `prefab-environs-454005-v7.aditi7890.sales`
    LIMIT 10
"""

# Run query and get results as DataFrame
df = client.query(query).to_dataframe()
print(df)


C:\Users\Abhishek Sinha\AppData\Roaming\Python\Python311\site-packages\google\cloud\bigquery\table.py:1933: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


         Date                 Timestamp Invoice_No_   Payment_Type Order_Type  \
0  2024-10-13 2024-10-13 22:42:25+00:00       11507           Cash    Dine In   
1  2024-10-04 2024-10-04 18:53:25+00:00       C1984           Cash    Dine In   
2  2025-03-10 2025-03-11 00:24:33+00:00       21337  Other [Paytm]    Dine In   
3  2024-08-24 2024-08-24 15:23:25+00:00        8455           CARD    Dine In   
4  2025-01-20 2025-01-20 22:12:29+00:00       18290           CARD    Dine In   
5  2024-08-26 2024-08-26 22:01:54+00:00        8642  Other [Paytm]    Pick Up   
6  2025-02-09 2025-02-09 14:47:06+00:00       19595           Cash    Dine In   
7  2024-06-17 2024-06-17 21:49:00+00:00        4571           CARD    Dine In   
8  2024-10-23 2024-10-23 22:21:06+00:00       12132   Other [AMEX]    Dine In   
9  2024-08-16 2024-08-16 23:37:14+00:00        8028           CARD    Dine In   

           Area                                    Item_Name  Price  Qty_  \
0  Garden Table                

In [3]:
import matplotlib.pyplot as plt
import numpy as np
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image, PageBreak
from reportlab.graphics.shapes import Drawing
from reportlab.graphics.charts.piecharts import Pie
from reportlab.graphics.charts.barcharts import VerticalBarChart
from reportlab.graphics import renderPM
from io import BytesIO
import os
from matplotlib.figure import Figure
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import matplotlib
matplotlib.use('Agg')

def create_audit_report(output_filename, data):
    doc = SimpleDocTemplate(output_filename, pagesize=letter, leftMargin=0.5*inch, rightMargin=0.5*inch, topMargin=0.5*inch, bottomMargin=0.5*inch)
    styles = getSampleStyleSheet()
    elements = []
    
    # Create a custom style for the main title
    title_style = ParagraphStyle(
        'Title',
        parent=styles['Heading1'],
        fontSize=36,
        alignment=1,
        spaceAfter=10,
        textColor=colors.Color(0.2, 0.3, 0.4)
    )
    
    subtitle_style = ParagraphStyle(
        'Subtitle',
        parent=styles['Heading2'],
        fontSize=24,
        alignment=1,
        spaceAfter=20,
        textColor=colors.Color(0.5, 0.5, 0.5)
    )
    
    section_title_style = ParagraphStyle(
        'SectionTitle',
        parent=styles['Heading3'],
        fontSize=16,
        spaceAfter=10,
        textColor=colors.Color(0.2, 0.3, 0.4)
    )
    
    body_text_style = ParagraphStyle(
        'BodyText',
        parent=styles['Normal'],
        fontSize=11,
        leading=14,
        spaceAfter=6
    )
    
    recommendation_style = ParagraphStyle(
        'Recommendation',
        parent=styles['Normal'],
        fontSize=12,
        leading=14,
        leftIndent=20,
        bulletIndent=10,
        spaceAfter=10,
        textColor=colors.Color(0.2, 0.3, 0.4)
    )
    
    recommendation_header_style = ParagraphStyle(
        'RecommendationHeader',
        parent=styles['Normal'],
        fontSize=13,
        leading=16,
        fontName='Helvetica-Bold',
        textColor=colors.Color(0.2, 0.3, 0.4),
        spaceAfter=8
    )
    
    # First Page - Summary
    elements.append(Paragraph("AUDIT REPORT", title_style))
    elements.append(Paragraph("SUMMARY", subtitle_style))
    
    # Add horizontal line
    t = Table([[""], [""]], colWidths=[7*inch], rowHeights=[1, 1])
    t.setStyle(TableStyle([
        ('LINEBELOW', (0, 0), (0, 0), 1, colors.gray),
    ]))
    elements.append(t)
    elements.append(Spacer(1, 0.1*inch))
    
    # Anomalies flagged section
    anomalies_percent = (data["anomalies_flagged"] / data["total_transactions"]) * 100
    
    # Create the main table layout for anomalies and percentage
    anomalies_data = Paragraph(f"<font size='38' color='#2B3F53'>{data['anomalies_flagged']}</font>", styles["Normal"])
    anomalies_text = Paragraph("<font size='16' color='#5A6A7A'>ANOMALIES<br/><br/>FLAGGED</font>", styles["Normal"])
    
    # Create donut chart for percentage
    donut_img = BytesIO()
    fig = Figure(figsize=(3, 3))
    canvas = FigureCanvas(fig)
    ax = fig.add_subplot(111)
    
    # Create a donut chart
    size = 0.3
    vals = [anomalies_percent, 100 - anomalies_percent]
    colors_pie = ['#2B3F53', '#E6E9ED']
    
    ax.pie(vals, radius=1, colors=colors_pie, wedgeprops=dict(width=size, edgecolor='w'))
    # Add percentage text in center
    ax.text(0, 0, f"{int(anomalies_percent)}%", ha='center', va='center', fontsize=40, fontweight='bold')
    ax.set_aspect('equal')
    ax.axis('off')
    
    fig.savefig(donut_img, format='png', transparent=True, bbox_inches='tight')
    donut_img.seek(0)
    donut_chart = Image(donut_img, width=2.5*inch, height=2.5*inch)
    
    # Add "OF TOTAL" text
    percent_text = Paragraph("<font size='16' color='#5A6A7A'>OF TOTAL</font>", styles["Normal"])
    
    # Create two-column layout
    anomalies_table = Table([
        [anomalies_data, donut_chart],
        [anomalies_text, percent_text]
    ], colWidths=[3.5*inch, 3.5*inch])
    
    anomalies_table.setStyle(TableStyle([
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('ALIGN', (0, 0), (0, -1), 'LEFT'),
        ('ALIGN', (1, 0), (1, -1), 'CENTER'),
    ]))
    
    elements.append(anomalies_table)
    elements.append(Spacer(1, 0.2*inch))
    
    # Add horizontal line
    t = Table([[""], [""]], colWidths=[7*inch], rowHeights=[1, 1])
    t.setStyle(TableStyle([
        ('LINEBELOW', (0, 0), (0, 0), 1, colors.gray),
    ]))
    elements.append(t)
    elements.append(Spacer(1, 0.2*inch))
    
    # Create vertical bar chart for types of anomalies
    bar_img = BytesIO()
    fig = Figure(figsize=(3, 2))
    canvas = FigureCanvas(fig)
    ax = fig.add_subplot(111)
    
    anomaly_types = list(data["types_of_anomalies"].keys())
    anomaly_values = list(data["types_of_anomalies"].values())
    
    # Sort anomalies for the bar chart display
    sorted_indices = np.argsort(anomaly_values)[::-1]  # Sort in descending order
    sorted_types = [anomaly_types[i] for i in sorted_indices]
    sorted_values = [anomaly_values[i] for i in sorted_indices]
    
    # Create bar chart
    y_pos = np.arange(len(sorted_types))
    colors_bar = ['#243A5E', '#344B6F', '#445C80', '#546D91']
    bars = ax.bar(y_pos, sorted_values, color=colors_bar[:len(sorted_types)])
    
    ax.set_xticks([])
    ax.set_yticks([])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    
    fig.savefig(bar_img, format='png', transparent=True, bbox_inches='tight')
    bar_img.seek(0)
    bar_chart = Image(bar_img, width=3*inch, height=2*inch)
    
    # Create server performance bars
    server_img = BytesIO()
    fig = Figure(figsize=(3, 1.8))
    canvas = FigureCanvas(fig)
    ax = fig.add_subplot(111)
    
    servers = list(data["server_performance"].keys())
    performance_values = list(data["server_performance"].values())
    
    # Sort servers by performance
    sorted_indices = np.argsort(performance_values)[::-1]  # Sort in descending order
    sorted_servers = [servers[i] for i in sorted_indices]
    sorted_perf_values = [performance_values[i] for i in sorted_indices]
    
    # Create horizontal bars
    y_pos = np.arange(len(sorted_servers))
    bars = ax.barh(y_pos, sorted_perf_values, color='#324D6A', height=0.4)
    
    ax.set_yticks([])
    ax.set_xticks([])
    ax.axis('off')
    
    fig.tight_layout()
    fig.savefig(server_img, format='png', transparent=True, bbox_inches='tight')
    server_img.seek(0)
    server_chart = Image(server_img, width=3*inch, height=1.8*inch)
    
    # Two-column layout for Types of Anomalies and Server Performance
    performance_table = Table([
        [Paragraph("Types of Anomalies", section_title_style), Paragraph("Server Performance", section_title_style)],
        [bar_chart, server_chart]
    ], colWidths=[3.5*inch, 3.5*inch])
    
    performance_table.setStyle(TableStyle([
        ('VALIGN', (0, 0), (-1, -1), 'TOP'),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
    ]))
    
    elements.append(performance_table)
    
    # Add horizontal line
    t = Table([[""], [""]], colWidths=[7*inch], rowHeights=[1, 1])
    t.setStyle(TableStyle([
        ('LINEBELOW', (0, 0), (0, 0), 1, colors.gray),
    ]))
    elements.append(t)
    
    # Page break before detailed pages
    elements.append(PageBreak())
    
    # Detailed pages
    elements.append(Paragraph("DETAILED AUDIT FINDINGS", title_style))
    elements.append(Spacer(1, 0.2*inch))
    
    # Types of Anomalies - Detailed Analysis
    elements.append(Paragraph("Types of Anomalies - Detailed Analysis", section_title_style))
    elements.append(Spacer(1, 0.1*inch))
    
    # Create pie chart for anomaly types
    pie_img = BytesIO()
    fig = Figure(figsize=(3.5, 3.5))
    canvas = FigureCanvas(fig)
    ax = fig.add_subplot(111)
    
    # Calculate percentages for pie chart labels
    total_anomalies = sum(anomaly_values)
    percentages = [value / total_anomalies * 100 for value in anomaly_values]
    labels = [f"{anomaly_types[i]}\n{percentages[i]:.1f}%" for i in range(len(anomaly_types))]
    
    colors_pie = ['#243A5E', '#344B6F', '#445C80', '#546D91']
    ax.pie(anomaly_values, labels=None, colors=colors_pie, shadow=False, startangle=90)
    
    # Add percentage labels manually
    for i, p in enumerate(ax.patches):
        ang = (p.theta2 - p.theta1) / 2. + p.theta1
        x = np.cos(np.deg2rad(ang))
        y = np.sin(np.deg2rad(ang))
        if i < len(labels):
            ax.annotate(labels[i], xy=(0.7*x, 0.7*y), ha='center', va='center', fontsize=10)
    
    ax.axis('equal')
    
    fig.savefig(pie_img, format='png', transparent=True, bbox_inches='tight')
    pie_img.seek(0)
    pie_chart = Image(pie_img, width=3.5*inch, height=3.5*inch)
    
    # Table with anomaly statistics
    anomaly_table_data = [
        ["Anomaly Type", "Count", "Percentage"],
    ]
    
    # Sort anomalies for the table
    sorted_indices = np.argsort(anomaly_values)[::-1]  # Sort in descending order
    sorted_types = [anomaly_types[i] for i in sorted_indices]
    sorted_values = [anomaly_values[i] for i in sorted_indices]
    
    for i, anomaly_type in enumerate(sorted_types):
        count = sorted_values[i]
        percentage = (count / total_anomalies) * 100
        anomaly_table_data.append([anomaly_type, str(count), f"{percentage:.1f}%"])
    
    anomaly_table = Table(anomaly_table_data, colWidths=[1.5*inch, 1*inch, 1*inch])
    anomaly_table.setStyle(TableStyle([
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('ALIGN', (1, 0), (-1, -1), 'CENTER'),
        ('LINEBELOW', (0, 0), (-1, 0), 1, colors.gray),
        ('BACKGROUND', (0, 0), (-1, 0), colors.whitesmoke),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('LINEBELOW', (0, 1), (-1, -1), 0.5, colors.lightgrey),
        ('TOPPADDING', (0, 0), (-1, -1), 8),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 8),
    ]))
    
    # Layout for pie chart and table side by side
    anomaly_detail_table = Table([
        [pie_chart, anomaly_table]
    ], colWidths=[3.5*inch, 3.5*inch])
    
    anomaly_detail_table.setStyle(TableStyle([
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('ALIGN', (0, 0), (0, 0), 'CENTER'),
    ]))
    
    elements.append(anomaly_detail_table)
    
    # Add page break
    elements.append(PageBreak())
    
    # Server Performance - Detailed Analysis
    elements.append(Paragraph("Server Performance - Detailed Analysis", section_title_style))
    elements.append(Spacer(1, 0.1*inch))
    
    # Create horizontal bar chart for server performance
    server_detail_img = BytesIO()
    fig = Figure(figsize=(6, 3))
    canvas = FigureCanvas(fig)
    ax = fig.add_subplot(111)
    
    servers = list(data["server_performance"].keys())
    performance_values = list(data["server_performance"].values())
    
    # Sort servers by performance
    sorted_indices = np.argsort(performance_values)
    sorted_servers = [servers[i] for i in sorted_indices]
    sorted_perf_values = [performance_values[i] for i in sorted_indices]
    
    bars = ax.barh(sorted_servers, sorted_perf_values, color='#324D6A')
    ax.set_xlabel('Number of Anomalies')
    ax.set_title('Server Anomalies')
    
    # Add counts at the end of each bar
    for i, v in enumerate(sorted_perf_values):
        ax.text(v + 0.1, i, str(v), va='center')
    
    fig.tight_layout()
    fig.savefig(server_detail_img, format='png', transparent=True)
    server_detail_img.seek(0)
    server_detail_chart = Image(server_detail_img, width=6.5*inch, height=3*inch)
    
    elements.append(server_detail_chart)
    elements.append(Spacer(1, 0.1*inch))
    
    # Server performance table
    server_table_data = [
        ["Server Name", "Anomalies", "Percentage of Total"],
    ]
    
    total_server_anomalies = sum(performance_values)
    
    # Sort for table display
    sorted_indices = np.argsort(performance_values)[::-1]  # Descending order
    sorted_servers = [servers[i] for i in sorted_indices]
    sorted_perf_values = [performance_values[i] for i in sorted_indices]
    
    for i, server in enumerate(sorted_servers):
        count = sorted_perf_values[i]
        percentage = (count / total_server_anomalies) * 100
        server_table_data.append([server, str(count), f"{percentage:.1f}%"])
    
    server_table = Table(server_table_data, colWidths=[2.5*inch, 2*inch, 2*inch])
    server_table.setStyle(TableStyle([
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('ALIGN', (1, 0), (-1, -1), 'CENTER'),
        ('LINEBELOW', (0, 0), (-1, 0), 1, colors.gray),
        ('BACKGROUND', (0, 0), (-1, 0), colors.whitesmoke),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('LINEBELOW', (0, 1), (-1, -1), 0.5, colors.lightgrey),
        ('TOPPADDING', (0, 0), (-1, -1), 8),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 8),
    ]))
    
    elements.append(server_table)
    
    # Add page break
    elements.append(PageBreak())
    
    # Top Restaurants - Detailed Analysis
    elements.append(Paragraph("Top 5 Restaurants with Anomalies - Detailed Analysis", section_title_style))
    elements.append(Spacer(1, 0.1*inch))
    
    # Create horizontal bar chart for top restaurants
    rest_img = BytesIO()
    fig = Figure(figsize=(6, 3))
    canvas = FigureCanvas(fig)
    ax = fig.add_subplot(111)
    
    restaurant_names = [item[1] for item in data["top_restaurants"]]
    restaurant_counts = [int(item[2]) for item in data["top_restaurants"]]
    
    # Horizontal bar chart for restaurants
    bars = ax.barh(restaurant_names, restaurant_counts, color='#324D6A')
    ax.set_xlabel('Number of Anomalies')
    ax.set_title('Top 5 Restaurants with Anomalies')
    
    # Add counts at the end of each bar
    for i, v in enumerate(restaurant_counts):
        ax.text(v + 0.1, i, str(v), va='center')
    
    fig.tight_layout()
    fig.savefig(rest_img, format='png', transparent=True)
    rest_img.seek(0)
    rest_chart = Image(rest_img, width=6.5*inch, height=3*inch)
    
    elements.append(rest_chart)
    elements.append(Spacer(1, 0.1*inch))
    
    # Detailed restaurant table
    rest_table_data = [
        ["Rank", "Restaurant Name", "Anomalies", "Percentage of Total"],
    ]
    
    total_rest_anomalies = sum(restaurant_counts)
    for rank, name, count in data["top_restaurants"]:
        percentage = (int(count) / total_rest_anomalies) * 100
        rest_table_data.append([rank, name, count, f"{percentage:.1f}%"])
    
    rest_table = Table(rest_table_data, colWidths=[0.75*inch, 2.75*inch, 1.5*inch, 1.5*inch])
    rest_table.setStyle(TableStyle([
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('ALIGN', (0, 0), (0, -1), 'CENTER'),
        ('ALIGN', (2, 0), (-1, -1), 'CENTER'),
        ('LINEBELOW', (0, 0), (-1, 0), 1, colors.gray),
        ('BACKGROUND', (0, 0), (-1, 0), colors.whitesmoke),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('LINEBELOW', (0, 1), (-1, -1), 0.5, colors.lightgrey),
        ('TOPPADDING', (0, 0), (-1, -1), 8),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 8),
    ]))
    
    elements.append(rest_table)
    
    # Add page break for cash discrepancy analysis
    elements.append(PageBreak())
    
    # Cash Discrepancy - Detailed Analysis
    elements.append(Paragraph("Cash Discrepancy - Detailed Analysis", section_title_style))
    elements.append(Spacer(1, 0.1*inch))
    
    # Create horizontal bar chart for cash discrepancy
    cash_img = BytesIO()
    fig = Figure(figsize=(6, 2))
    canvas = FigureCanvas(fig)
    ax = fig.add_subplot(111)
    
    cash_categories = list(data["cash_discrepancy"].keys())
    cash_values = list(data["cash_discrepancy"].values())
    
    # Sort for visualization
    sorted_indices = np.argsort(cash_values)[::-1]
    sorted_cash_categories = [cash_categories[i] for i in sorted_indices]
    sorted_cash_values = [cash_values[i] for i in sorted_indices]
    
    colors_cash = ['#8B0000', '#006400']  # Red for Short, Green for Over
    color_map = {
        'Short': '#8B0000',
        'Over': '#006400'
    }
    
    bar_colors = [color_map.get(category, '#324D6A') for category in sorted_cash_categories]
    
    bars = ax.barh(sorted_cash_categories, sorted_cash_values, color=bar_colors)
    ax.set_xlabel('Amount')
    ax.set_title('Cash Discrepancy')
    
    # Add values at the end of each bar
    for i, v in enumerate(sorted_cash_values):
        ax.text(v + 0.1, i, str(v), va='center')
    
    fig.tight_layout()
    fig.savefig(cash_img, format='png', transparent=True)
    cash_img.seek(0)
    cash_chart = Image(cash_img, width=6.5*inch, height=2*inch)
    
    elements.append(cash_chart)
    elements.append(Spacer(1, 0.1*inch))
    
    # Cash discrepancy table
    cash_table_data = [
        ["Category", "Count", "Percentage of Total"],
    ]
    
    total_cash_discrepancy = sum(cash_values)
    for category, value in data["cash_discrepancy"].items():
        percentage = (value / total_cash_discrepancy) * 100
        cash_table_data.append([category, str(value), f"{percentage:.1f}%"])
    
    cash_table = Table(cash_table_data, colWidths=[2.5*inch, 2*inch, 2*inch])
    cash_table.setStyle(TableStyle([
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('ALIGN', (1, 0), (-1, -1), 'CENTER'),
        ('LINEBELOW', (0, 0), (-1, 0), 1, colors.gray),
        ('BACKGROUND', (0, 0), (-1, 0), colors.whitesmoke),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('LINEBELOW', (0, 1), (-1, -1), 0.5, colors.lightgrey),
        ('TOPPADDING', (0, 0), (-1, -1), 8),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 8),
    ]))
    
    elements.append(cash_table)
    
    # Add page break for recommendations section
    elements.append(PageBreak())
    
    # Recommendations Section
    elements.append(Paragraph("RECOMMENDATIONS", title_style))
    elements.append(Spacer(1, 0.2*inch))
    
    # Add executive summary
    if "recommendations_summary" in data:
        elements.append(Paragraph("Executive Summary", section_title_style))
        elements.append(Paragraph(data["recommendations_summary"], body_text_style))
        elements.append(Spacer(1, 0.2*inch))
    
    # Add horizontal line
    t = Table([[""], [""]], colWidths=[7*inch], rowHeights=[1, 1])
    t.setStyle(TableStyle([
        ('LINEBELOW', (0, 0), (0, 0), 1, colors.gray),
    ]))
    elements.append(t)
    elements.append(Spacer(1, 0.2*inch))
    
    # Add recommendation categories
    if "recommendations" in data:
        for category, recommendations in data["recommendations"].items():
            # Create a box with a colored header for each category
            category_header = Table(
                [[Paragraph(category, recommendation_header_style)]], 
                colWidths=[7*inch], 
                rowHeights=[0.3*inch]
            )
            
            category_header.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (0, 0), colors.Color(0.2, 0.3, 0.4, 0.1)),
                ('VALIGN', (0, 0), (0, 0), 'MIDDLE'),
                ('LEFTPADDING', (0, 0), (0, 0), 10),
                ('BOTTOMPADDING', (0, 0), (0, 0), 5),
                ('TOPPADDING', (0, 0), (0, 0), 5),
            ]))
            
            elements.append(category_header)
            elements.append(Spacer(1, 0.1*inch))
            
            # Add individual recommendations as bullet points
            for i, recommendation in enumerate(recommendations):
                bullet_text = f"• {recommendation}"
                elements.append(Paragraph(bullet_text, recommendation_style))
            
            elements.append(Spacer(1, 0.2*inch))
    
    # Add priority action items if provided
    # Priority action items table with adjusted column widths
    # Priority action items table with text wrapping for cells
    if "priority_actions" in data:
        elements.append(Paragraph("Priority Action Items", section_title_style))
        elements.append(Spacer(1, 0.1*inch))
        
        # Create custom style for table cell text
        cell_style = ParagraphStyle(
            'CellText',
            parent=styles['Normal'],
            fontSize=10,
            leading=12,  # Line spacing
            wordWrap='CJK',  # Enable word wrapping
            alignment=4  # Left alignment
        )
        
        # Prepare header row
        action_table_data = [
            ["Priority", "Action Item", "Responsible", "Timeline"],
        ]
        
        # Process data and wrap text in Paragraph objects
        for priority, action, responsible, timeline in data["priority_actions"]:
            # Wrap each cell in a Paragraph object to enable text wrapping
            wrapped_priority = Paragraph(priority, cell_style)
            wrapped_action = Paragraph(action, cell_style)
            wrapped_responsible = Paragraph(responsible, cell_style)
            wrapped_timeline = Paragraph(timeline, cell_style)
            
            action_table_data.append([wrapped_priority, wrapped_action, wrapped_responsible, wrapped_timeline])
        
        # Adjust column widths to prevent text overlap
        action_table = Table(action_table_data, colWidths=[0.7*inch, 2.8*inch, 2.0*inch, 1.5*inch])
        action_table.setStyle(TableStyle([
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
            ('ALIGN', (0, 0), (0, -1), 'CENTER'),
            ('LINEBELOW', (0, 0), (-1, 0), 1, colors.gray),
            ('BACKGROUND', (0, 0), (-1, 0), colors.whitesmoke),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('LINEBELOW', (0, 1), (-1, -1), 0.5, colors.lightgrey),
            ('TOPPADDING', (0, 0), (-1, -1), 8),
            ('BOTTOMPADDING', (0, 0), (-1, -1), 8),
            # Allow more space for rows to accommodate wrapped text
            ('MINROWHEIGHT', (0, 1), (-1, -1), 30),
        ]))
        
        elements.append(action_table)
    
    # Build the document
    doc.build(elements)
    


def prepare_audit_data(df):
    # Initialize with static data as fallback
    static_data = {
        "total_transactions": 1000,
        "anomalies_flagged": 120,
        "types_of_anomalies": {
            "Late Orders": 40, 
            "Wrong Bills": 25, 
            "Missing Items": 30, 
            "Manual Override": 25,
            "Incorrect Pricing": 18,
            "Unauthorized Comps": 15,
            "Excessive Voids": 12,
            "Multiple Swipes": 10
        },
        "server_performance": {
            "John": 15, 
            "Emma": 10, 
            "Alex": 5, 
            "Sara": 2,
            "Michael": 20,
            "Jessica": 8,
            "David": 12,
            "Amanda": 6
        },
        "cash_discrepancy": {
            "Short": 18, 
            "Over": 7,
            "Missing Receipt": 12,
            "Incorrect Change": 5
        },
        "top_restaurants": [
            ["#1", "Foodie's Corner", "25"],
            ["#2", "Spice Hub", "21"],
            ["#3", "Grill Master", "18"],
            ["#4", "Urban Bites", "15"],
            ["#5", "Tandoori Treats", "13"]
        ],
        "recommendations_summary": "Based on our audit findings, we identified several areas requiring immediate attention. The recommendations below address the most critical issues that will improve operational efficiency, reduce cash discrepancies, and enhance overall service quality.",
        "recommendations": {
            "Staff Training": [
                "Implement bi-weekly training sessions for servers with high anomaly rates",
                "Develop a comprehensive POS system training manual for new employees",
                "Create a certification program for cash handling procedures"
            ],
            "Process Improvement": [
                "Review and optimize order fulfillment workflow to reduce late orders",
                "Implement a double-check system for bills over $100",
                "Create a standardized procedure for handling manual overrides"
            ]
        },
        "priority_actions": [
            ["High", "Implement server training program focusing on top offenders", "Training Manager", "30 days"],
            ["High", "Fix POS system issues for Multiple Swipes and Manual Overrides", "IT Department", "14 days"],
            ["Medium", "Implement daily cash drawer audits", "Floor Managers", "45 days"]
        ]
    }
    
    # Initialize the data dictionary with static data
    data = static_data.copy()
    
    # Now override with actual data from DataFrame where available
    
    # 1. Calculate total transactions from Invoice numbers
    if 'Invoice_No_' in df.columns:
        data["total_transactions"] = len(df['Invoice_No_'].unique())
    
    # 2. Identify anomalies from the data
    data["types_of_anomalies"] = {}
    
    # Late Orders - if preparation time data exists
    if 'Food_Preparation_Time_Z' in df.columns or 'Food_Preparation_Time_S' in df.columns:
        prep_time_col = 'Food_Preparation_Time_Z' if 'Food_Preparation_Time_Z' in df.columns else 'Food_Preparation_Time_S'
        late_orders = df[df[prep_time_col] > 45]  # 45 minutes threshold
        data["types_of_anomalies"]["Late Orders"] = len(late_orders['Invoice_No_'].unique())
    elif 'Late Orders' in static_data["types_of_anomalies"]:
        data["types_of_anomalies"]["Late Orders"] = static_data["types_of_anomalies"]["Late Orders"]
    
    # Cancelled Orders
    if 'Status' in df.columns:
        cancelled_orders = df[df['Status'].str.lower().str.contains('cancel', na=False)]
        data["types_of_anomalies"]["Cancelled Orders"] = len(cancelled_orders['Invoice_No_'].unique())
    elif 'Cancelled Orders' in static_data["types_of_anomalies"]:
        data["types_of_anomalies"]["Cancelled Orders"] = static_data["types_of_anomalies"]["Cancelled Orders"]
    
    # High Discounts
    if 'Discount' in df.columns and 'Sub_Total' in df.columns:
        high_discounts = df[df['Discount'] > (0.2 * df['Sub_Total'])]  # >20% discount
        data["types_of_anomalies"]["High Discounts"] = len(high_discounts['Invoice_No_'].unique())
    elif 'High Discounts' in static_data["types_of_anomalies"]:
        data["types_of_anomalies"]["High Discounts"] = static_data["types_of_anomalies"]["High Discounts"]
    
    # Manual Overrides
    if 'Assign_To' in df.columns:
        manual_overrides = df[df['Assign_To'] != 'System']
        data["types_of_anomalies"]["Manual Overrides"] = len(manual_overrides['Invoice_No_'].unique())
    elif 'Manual Override' in static_data["types_of_anomalies"]:
        data["types_of_anomalies"]["Manual Override"] = static_data["types_of_anomalies"]["Manual Override"]
    
    # For any remaining anomaly types from static data that we haven't processed
    for anomaly_type, count in static_data["types_of_anomalies"].items():
        if anomaly_type not in data["types_of_anomalies"]:
            data["types_of_anomalies"][anomaly_type] = count
    
    # 3. Server Performance
    if 'Server_Name' in df.columns and 'Status' in df.columns:
        server_anomalies = df[df['Status'].str.lower().str.contains('cancel', na=False)]['Server_Name'].value_counts().to_dict()
        if server_anomalies:  # Only use if we found data
            data["server_performance"] = server_anomalies
    
    # 4. Top Restaurants (Outlets)
    if 'Outlet_Name_z' in df.columns and 'Status' in df.columns:
        outlet_anomalies = df[df['Status'].str.lower().str.contains('cancel', na=False)]['Outlet_Name_z'].value_counts().head(5)
        if not outlet_anomalies.empty:
            data["top_restaurants"] = [
                [f"#{i+1}", outlet, str(count)] 
                for i, (outlet, count) in enumerate(outlet_anomalies.items())
            ]
    
    # 5. Cash Discrepancy - harder to detect from raw data, so we'll mostly use static
    # (This would typically require specific reconciliation data)
    
    # Calculate total anomalies flagged
    data["anomalies_flagged"] = sum(data["types_of_anomalies"].values())
    
    return data


def generate_audit_report_from_dataframe(df, output_filename="restaurant_audit_report.pdf"):
    """
    Generate an audit report from a DataFrame, using static data where real data isn't available.
    
    Args:
        df (pd.DataFrame): DataFrame containing the restaurant transaction data
        output_filename (str): Path/filename for the output PDF report
    """
    # Prepare the data for the report
    audit_data = prepare_audit_data(df)
    
    # Generate the report using the existing function
    create_audit_report(output_filename, audit_data)
    
    print(f"Audit report generated successfully at: {output_filename}")

sample_data = {
        "total_transactions": 1000,
        "anomalies_flagged": 120,
        "types_of_anomalies": {
            "Late Orders": 40, 
            "Wrong Bills": 25, 
            "Missing Items": 30, 
            "Manual Override": 25,
            "Incorrect Pricing": 18,
            "Unauthorized Comps": 15,
            "Excessive Voids": 12,
            "Multiple Swipes": 10
        },
        "server_performance": {
            "John": 15, 
            "Emma": 10, 
            "Alex": 5, 
            "Sara": 2,
            "Michael": 20,
            "Jessica": 8,
            "David": 12,
            "Amanda": 6
        },
        "cash_discrepancy": {
            "Short": 18, 
            "Over": 7,
            "Missing Receipt": 12,
            "Incorrect Change": 5
        },
        "top_restaurants": [
            ["#1", "Foodie's Corner", "25"],
            ["#2", "Spice Hub", "21"],
            ["#3", "Grill Master", "18"],
            ["#4", "Urban Bites", "15"],
            ["#5", "Tandoori Treats", "13"]
        ],
        "recommendations_summary": "Based on our audit findings, we identified several areas requiring immediate attention. The recommendations below address the most critical issues that will improve operational efficiency, reduce cash discrepancies, and enhance overall service quality.",
        "recommendations": {
            "Staff Training": [
                "Implement bi-weekly training sessions for servers with high anomaly rates",
                "Develop a comprehensive POS system training manual for new employees",
                "Create a certification program for cash handling procedures"
            ],
            "Process Improvement": [
                "Review and optimize order fulfillment workflow to reduce late orders",
                "Implement a double-check system for bills over $100",
                "Create a standardized procedure for handling manual overrides"
            ]
        },
        "priority_actions": [
            ["High", "Implement server training program focusing on top offenders", "Training Manager", "30 days"],
            ["High", "Fix POS system issues for Multiple Swipes and Manual Overrides", "IT Department", "14 days"],
            ["Medium", "Implement daily cash drawer audits", "Floor Managers", "45 days"]
        ]
    }

    
create_audit_report("restaurant_audit_report.pdf", sample_data)
print("Audit report generated successfully!")

Audit report generated successfully!
